In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from os.path import join
from glob import glob
from io import StringIO
from textwrap import dedent
import pickle

import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import defaultdict

from src.single_sided_network_v2 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model

from doit.tools import register_doit_as_IPython_magic
import clusterjob

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2018-04-14_mg_oct_single_excitation.json',
}
register_doit_as_IPython_magic()

In [4]:
clusterjob.JobScript.read_defaults('./config/mlhpc_cluster.ini')

In [5]:
! qdyn_version

QDYN 2.0dev revision dfd21683b898e790985b519daac35aeef7721f6c (master)
  features: no-check-cheby, no-check-newton, no-parallel-ham, use-mkl=sequential, use-mpi=intel, parallel-oct, backtraces, no-debug, no-no-ipo
  compiled with ifort on Mon Nov 13 14:54:23 2017 on host mlhpc2


$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Compare variants of Krotov's method for optimization of a large-N Dicke state

In [6]:
ROOT = './data/oct_single_excitation/'

In [7]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    nt = row['nt']
    if frac_part == '00':
        if int(nt) == 500:
            T_str = "%s" % int_part
        else:
            T_str = "%s_nt%s" % (int_part, nt)
    else:
        if int(nt) == 500:
            T_str = '%s_%s' % (int_part, frac_part)
        else:
            T_str = '%s_%s_nt%s' % (int_part, frac_part, nt)
    rf = "%02dnodesT%s_%s_ntrajs%s" % (
        row['nodes'], T_str, row['variant'], row['n_trajs'])
    return join(ROOT, rf)

## SLH Models

For very large networks, the SLH models take a significant amount of time to calculate, so we'll want to cache them.

In [8]:
SLH_MODELS = {}

In [9]:
%mkdir -p {ROOT}

In [10]:
SLH_PICKLE_FILE = join(ROOT, "slh_models.pickle")

In [11]:
if os.path.isfile(SLH_PICKLE_FILE):
    with open(SLH_PICKLE_FILE, 'rb') as pickle_fh:
        SLH_MODELS.update(pickle.load(pickle_fh))

In [12]:
SLH_MODELS.keys()

dict_keys([50, 20, 10])

## action wrappers

In [13]:
from src.doit_actions_v2 import (
    update_config, wait_for_clusterjob, run_traj_prop,
    write_rho_prop_custom_config, collect_rho_prop_errors)

In [14]:
def submit_optimization(rf, n_trajs, variant, task):
    """Asynchronously run optimization, using either the
    standard QDYN implementation (variant = 'normal') or the 'crossoct' QDYN
    implementation at a particular revision (variant='cross')

    Does not support use of MPI, so this should only run on a workstation with
    more cores than the total number of trajectories
    """
    if os.path.isfile(join(rf, 'oct.job.dump')):
        return
    bodies = {
        'normal': dedent(r'''
            {module_load}
            cd $CLUSTERJOB_WORKDIR
            pwd
            OMP_NUM_THREADS={threads} \
              qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_ss .'''),
        'cross': dedent(r'''
            {module_load}
            module load prefix/crossoct

            qdyn_version | grep 66f6d24d2b07b9cc || {cancel_cmd}

            cd $CLUSTERJOB_WORKDIR
            pwd
            OMP_NUM_THREADS={threads} \
              qdyn_optimize --n-trajs={n_trajs} --J_T=J_T_cross .''')}
    body = bodies[variant]
    taskname = "oct_%s" % task.name.split(":")[-1]
    jobscript = clusterjob.JobScript(
        body=body, jobname=taskname, workdir=rf,
        stdout='oct.log')
    jobscript.n_trajs = str(int(n_trajs))
    jobscript.resources['ppn'] = 1 # not using MPI
    jobscript.resources['threads'] = min(int(n_trajs), 78)
    jobscript.threads = str(jobscript.resources['threads'])
    run = jobscript.submit(cache_id=taskname)
    run.dump(join(rf, 'oct.job.dump'))


## custom uptodate routines

In [15]:
from src.qdyn_model_v2 import pulses_uptodate

## OCT task definitions

In [ ]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    jobs = {}
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_nodes = row['nodes']
        nt = row['nt']
        try:
            slh = SLH_MODELS[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='open')
            SLH_MODELS[n_nodes] = slh
        if rf in jobs:
            continue
        mcwf = {  #   variant => whether to use MCWF
            'rho': False, 'nonherm': False, 'independent': True,
            'cross': True}
        jobs[rf] = {
            'name': str(rf),
            'actions': [
                # write the density matrix propagation data
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=nt,
                    kappa=1.0, E0_cycles=2,
                    mcwf=False, non_herm=False, # this defines rho-prop
                    single_excitation_subspace=True,
                    config='config_rho')),
                # write the data for the optimization
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=nt,
                    kappa=1.0, E0_cycles=2, mcwf=mcwf[row['variant']],
                    non_herm=(row['variant'] != 'rho'),
                    lambda_a=row['lambda_a'], keep_pulses='all',
                    iter_stop=int(row['iter_stop']), J_T_conv=row['J_T_conv'],
                    single_excitation_subspace=True,
                    config='config'))],
            'targets': [join(rf, 'config_rho'), join(rf, 'config')],
            'uptodate': [True, ] # up to date if targets exist
        }
    for job in jobs.values():
        yield job

In [ ]:
def task_update_runfolder():
    """For every row in params_df, update the config file in the appropriate
    runfolder with the value in that row
    (adjusting lambda_a, iter_stop and J_T_conv only)"""
    rf_jobs = defaultdict(list)
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        # we only update the config after any earlier optimization has finished
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        rf_jobs[rf].append(ind)
        yield {
            'name': str(ind),
            'actions': [
                (update_config, [], dict(
                    rf=rf, lambda_a=row['lambda_a'],
                    iter_stop=int(row['iter_stop']),
                    J_T_conv=row['J_T_conv']))],
            'file_dep': [join(rf, 'config')],
            'uptodate': [False, ],  # always run task
            'task_dep': task_dep}


In [ ]:
def task_submit_optimization():
    """Run optimization for every runfolder from params_df"""
    rf_jobs = defaultdict(list)
    optimization_variant = {
        'rho': 'normal',
        'nonherm': 'normal',
        'independent': 'normal',
        'cross': 'cross',
    }
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        task_dep = ['wait_for_optimization:%s' % ind2 for ind2 in rf_jobs[rf]]
        task_dep.append('update_runfolder:%s' % ind)
        yield {
            'name': str(ind),
            'actions': [
                (submit_optimization, [rf, ],
                 dict(n_trajs=row['n_trajs'],
                      variant=optimization_variant[row['variant']]))],
                # 'task' keyword arg is added automatically
            'task_dep': task_dep,
            'uptodate': [(pulses_uptodate, [], {'rf': rf}), ],
        }

In [ ]:
def task_wait_for_optimization():
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        yield {
            'name': str(ind),
            'task_dep': ['submit_optimization:%d' % ind],
            'actions': [
                (wait_for_clusterjob, [join(rf, 'oct.job.dump')], {}),]}

Additional tasks for evaluating the optimization success through a propagation
in Liouville space are defined farther below

##  OCT Submission

In [25]:
params_data_str = r'''
# nodes   T    nt  lambda_a      variant  n_trajs   iter_stop   J_T_conv
      2   5   500     0.001  rho                1        3000       1e-8
     50   5   500     0.001  rho                1        5000       1e-8
     50   5   500     0.001  independent        1        5000       1e-8
     50   5   500     0.001  independent        8        5000       1e-8
     50   5   500     0.001  cross              2        5000       1e-8
     50   5   500     0.001  cross              8        5000       1e-8
     50  20   500     0.001  rho                1        5000       1e-8
     50 100   500     0.001  rho                1        3000       1e-8
     50 200   500     0.001  rho                1        3000       1e-8
     50  20   500     0.001  independent        1        5000       1e-8
     50  20   500     0.001  independent        8        5000       1e-8
     50  20   500     0.001  cross              2        5000       1e-8
     50  20   500     0.001  cross              8        5000       1e-8
     20  20   500     0.001  rho                1        5000       1e-8
     20  20   500     0.001  independent        1        5000       1e-8
     20  20   500     0.001  independent        8        5000       1e-8
     20  50   500     0.001  independent        1        5000       1e-8
     20  50   500     0.001  independent        2        5000       1e-8
     20  50   500     0.001  independent        4        5000       1e-8
     20  50   500     0.001  independent        8        5000       1e-8
     20  50   500     0.001  independent       16        3000       1e-8
     20  50   500     0.001  independent       32        3000       1e-8
     20  50   500     0.001  independent       64        3000       1e-8
     20  50   500     0.001  independent      128        3000       1e-8
     20  20   500     0.001  cross              2        5000       1e-8
     20  50   500     0.001  cross              4        3000       1e-8
     20  50   500     0.001  cross              8        5000       1e-8
     20  50   500     0.001  cross             16        3000       1e-8
     20  50   500     0.001  cross             32        3000       1e-8
     20  50   500     0.001  cross             64        3000       1e-8
     20  50   500     0.001  cross            128        3000       1e-8
     20  25   500     0.001  rho                1        5000       1e-8
     20  50   500     0.001  rho                1        5000       1e-8
     20  50  1000     0.001  rho                1        5000       1e-8
     20 100   500     0.001  rho                1        5000       1e-8
     10  25   500     0.001  rho                1        5000       1e-8
     10  50   500     0.001  rho                1        5000       1e-8
'''
params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=[
            'nodes', 'T', 'nt', 'lambda_a', 'variant', 'n_trajs', 'iter_stop',
            'J_T_conv'],
        converters={
            'nodes': int, 'T': float, 'nt': int, 'lambda_a': float,
            'variant': str, 'n_trajs': int, 'iter_stop': int,
            'J_T_conv': float})


In [26]:
print(params_df.to_string())

    nodes      T    nt  lambda_a      variant  n_trajs  iter_stop      J_T_conv
0       2    5.0   500     0.001          rho        1       3000  1.000000e-08
1      50    5.0   500     0.001          rho        1       5000  1.000000e-08
2      50    5.0   500     0.001  independent        1       5000  1.000000e-08
3      50    5.0   500     0.001  independent        8       5000  1.000000e-08
4      50    5.0   500     0.001        cross        2       5000  1.000000e-08
5      50    5.0   500     0.001        cross        8       5000  1.000000e-08
6      50   20.0   500     0.001          rho        1       5000  1.000000e-08
7      50  100.0   500     0.001          rho        1       3000  1.000000e-08
8      50  200.0   500     0.001          rho        1       3000  1.000000e-08
9      50   20.0   500     0.001  independent        1       5000  1.000000e-08
10     50   20.0   500     0.001  independent        8       5000  1.000000e-08
11     50   20.0   500     0.001        

In [18]:
import logging
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(
    level=logging.DEBUG, filename='./data/oct_single_excitation.log')

In [ ]:
%doit -n 40 create_runfolder

In [ ]:
%doit -n 40 wait_for_optimization

In [ ]:
with open(SLH_PICKLE_FILE, 'wb') as pickle_fh:
    pickle.dump(SLH_MODELS, pickle_fh)

## Evaluate error exactly from density matrix propagation

In [54]:
RHO_PROP_ITERS = np.arange(6000, step=300)

In [55]:
def task_rho_prop_create_custom_config():
    """For every optimized pulse, at intermediate iteration numbers,
    write a custom config file the will propagate this pulse"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for oct_pulse_file in glob(join(rf, 'pulse1.oct.dat.0*')):
            oct_iter = int(os.path.splitext(oct_pulse_file)[-1][1:])
            if oct_iter not in RHO_PROP_ITERS:
                continue  # skip; we don't want to propagate *all* pulses
            assert os.path.isfile(oct_pulse_file.replace('pulse1', 'pulse2'))
            config_out = 'config_rho.%08d' % oct_iter
            yield {
                'name': str(rf) + "/%s" % config_out,
                'actions': [
                    (write_rho_prop_custom_config,
                     [rf, oct_iter, config_out])],
                'targets': [join(rf, config_out)],
                'uptodate': [True, ] # up to date if target exists
        }

In [56]:
def task_evaluate_rho_prop_error():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        for custom_config in glob(join(rf, 'config_rho.0*')):
            oct_iter = int(os.path.splitext(custom_config)[-1][1:])
            yield {
                'name': str(rf) + "/%s" % custom_config,
                'actions': [
                    (run_traj_prop, [rf, ], dict(
                        n_trajs=1, n_procs=None, use_oct_pulses=False,
                        config=os.path.split(custom_config)[-1]))],
                'targets': [join(rf, 'P_target.dat.%08d' % oct_iter)],
                'file_dep': [custom_config],
                'uptodate': [True, ] # up to date if target exists
        }

In [57]:
def task_collect_rho_prop_errors():
    """For every custom rho prop config file, perform the propagation"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        P_target_obs_files = glob(join(rf, 'P_target.dat.0*'))
        target_file = join(rf, 'rho_prop_error.dat')
        yield {
            'name': target_file,
            'actions': [
                (collect_rho_prop_errors, P_target_obs_files,
                 dict(outfile=target_file))],
            'targets': [target_file],
            'file_dep': P_target_obs_files,
            'uptodate': [False, ]
            # We always create a new file
        }

In [58]:
%doit -n 78 rho_prop_create_custom_config

-- rho_prop_create_custom_config:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00000000
-- rho_prop_create_custom_config:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00001500
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00000000
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00004500
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00001500
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00003000
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_independent_ntrajs1/config_rho.00000000
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_independent_ntrajs1/config_rho.00004500
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_independent_ntrajs1/config_rho.00001500
-- rho_prop_create_custom_config:./data/

.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_independent_ntrajs1/config_rho.00002100
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00003900
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_independent_ntrajs8/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00002100
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00001800
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/config_rho.00002700
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/config_rho.00000300
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_cross_ntrajs2/config_rho.00000300
-- rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00000000
.  rho_prop_create_custom_c

.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00000300
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs2/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00004800
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_independent_ntrajs8/config_rho.00000300
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs2/config_rho.00004200
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00003600
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00002400
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00000900
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_independent_ntrajs1/config_rho.00000000
-- rho_prop_c

.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.00000900
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs2/config_rho.00000600
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00001500
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00003000
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT5_cross_ntrajs8/config_rho.00001800
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_independent_ntrajs1/config_rho.00000900
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs1/config_rho.00004200
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs1/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs1/config_rho

-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00000000
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00004500
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_cross_ntrajs2/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs128/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_cross_ntrajs2/config_rho.00003600
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT20_cross_ntrajs2/config_rho.00002700
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs128/config_rho.00001800
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00001500
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00003000
.  rho_prop_create_cu

.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs64/config_rho.00000900
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_rho_ntrajs1/config_rho.00003900
.  rho_prop_create_custom_config:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00003300
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs16/config_rho.00001800
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00000900
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_cross_ntrajs4/config_rho.00000600
.  rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT50_rho_ntrajs1/config_rho.00002700
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT100_rho_ntrajs1/config_rho.00000000
-- rho_prop_create_custom_config:./data/oct_single_excitation/20nodesT100_rho_ntrajs1/config_rho.00004500
.  rho_prop_create_custom_config:./

In [59]:
%doit -n 78 evaluate_rho_prop_error

-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00000500
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00002000
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00002500
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00001000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00000500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00002000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation

-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00001200
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00003600
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/./data/oct_single_excitation/50nodesT100_rho_ntrajs1/config_rho.00003000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/./data/oct_single_excitation/50nodesT200_rho_ntrajs1/config_rho.00002400
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/./data/oct_single_excitation/50nodesT200_rho_ntrajs1/config_rho.00002700
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/./data/oct_single_excitation/50nodesT200_rho_ntrajs1/config_rho.00000000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT200_rho_ntrajs1/./

-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00001500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00004500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_cross_ntrajs8/./data/oct_single_excitation/50nodesT20_cross_ntrajs8/config_rho.00003000
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_rho_ntrajs1/./data/oct_single_excitation/20nodesT20_rho_ntrajs1/config_rho.00000000
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_rho_ntrajs1/./data/oct_single_excitation/20nodesT20_rho_ntrajs1/config_rho.00001500
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_rho_ntrajs1/./data/oct_single_excitation/20nodesT20_rho_ntrajs1/config_rho.00004500
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_rho_ntrajs1/./d

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs1/./data/oct_single_excitation/20nodesT20_independent_ntrajs1/config_rho.00003600
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.00002400
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.00002700
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.00004200
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.00004800
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT20_independent_ntrajs8/./data/oct_single_excitation/20nodesT20_independent_ntrajs8/config_rho.0000180

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00001800
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00002100
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00003900
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00003300
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.00000300
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs4/./data/oct_single_excitation/20nodesT50_independent_ntrajs4/config_rho.0000090

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs64/./data/oct_single_excitation/20nodesT50_independent_ntrajs64/config_rho.00002100
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs64/./data/oct_single_excitation/20nodesT50_independent_ntrajs64/config_rho.00000300
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs64/./data/oct_single_excitation/20nodesT50_independent_ntrajs64/config_rho.00000900
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs64/./data/oct_single_excitation/20nodesT50_independent_ntrajs64/config_rho.00000600
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs64/./data/oct_single_excitation/20nodesT50_independent_ntrajs64/config_rho.00001200
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_independent_ntrajs128/./data/oct_single_excitation/20nodesT50_independent_ntrajs128/conf

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00000900
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00004500
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00001200
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00000600
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00003600
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cross_ntrajs8/./data/oct_single_excitation/20nodesT50_cross_ntrajs8/config_rho.00003000
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_cro

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT25_rho_ntrajs1/./data/oct_single_excitation/20nodesT25_rho_ntrajs1/config_rho.00000600
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT25_rho_ntrajs1/./data/oct_single_excitation/20nodesT25_rho_ntrajs1/config_rho.00004500
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT25_rho_ntrajs1/./data/oct_single_excitation/20nodesT25_rho_ntrajs1/config_rho.00001200
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT25_rho_ntrajs1/./data/oct_single_excitation/20nodesT25_rho_ntrajs1/config_rho.00003600
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT25_rho_ntrajs1/./data/oct_single_excitation/20nodesT25_rho_ntrajs1/config_rho.00003000
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_rho_ntrajs1/./data/oct_single_excitation/20nodesT50_rho_ntrajs1/config_rho.00002400
.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT50_rho_ntrajs1/./data/oct_sing

.  evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT100_rho_ntrajs1/./data/oct_single_excitation/20nodesT100_rho_ntrajs1/config_rho.00003600
-- evaluate_rho_prop_error:./data/oct_single_excitation/20nodesT100_rho_ntrajs1/./data/oct_single_excitation/20nodesT100_rho_ntrajs1/config_rho.00003000
.  evaluate_rho_prop_error:./data/oct_single_excitation/10nodesT25_rho_ntrajs1/./data/oct_single_excitation/10nodesT25_rho_ntrajs1/config_rho.00002400
.  evaluate_rho_prop_error:./data/oct_single_excitation/10nodesT25_rho_ntrajs1/./data/oct_single_excitation/10nodesT25_rho_ntrajs1/config_rho.00002700
-- evaluate_rho_prop_error:./data/oct_single_excitation/10nodesT25_rho_ntrajs1/./data/oct_single_excitation/10nodesT25_rho_ntrajs1/config_rho.00000000
.  evaluate_rho_prop_error:./data/oct_single_excitation/10nodesT25_rho_ntrajs1/./data/oct_single_excitation/10nodesT25_rho_ntrajs1/config_rho.00004200
.  evaluate_rho_prop_error:./data/oct_single_excitation/10nodesT25_rho_ntrajs1/./data/oct_

In [60]:
%doit -n 78 collect_rho_prop_errors

-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00002500
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00002000
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00001000
-- evaluate_rho_prop_error:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/./data/oct_single_excitation/02nodesT5_rho_ntrajs1/config_rho.00000500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00004000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation/50nodesT5_rho_ntrajs1/config_rho.00002500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/./data/oct_single_excitation

-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00000000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00000300
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00003900
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00003000
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.00004500
-- evaluate_rho_prop_error:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/./data/oct_single_excitation/50nodesT20_independent_ntrajs1/config_rho.0000060

.  collect_rho_prop_errors:./data/oct_single_excitation/02nodesT5_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT5_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT5_independent_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT5_cross_ntrajs2/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT20_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT5_cross_ntrajs8/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT5_independent_ntrajs8/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT100_rho_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT20_independent_ntrajs1/rho_prop_error.dat
.  collect_rho_prop_errors:./data/oct_single_excitation/50nodesT20_independent_ntrajs8/rho_prop